In [6]:
import pytest
import json
from module.shift_creator import chain, WorkerShift, Shift, set_worker_info_data

ImportError: cannot import name 'chain' from 'module.shift_creator' (/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/module/shift_creator.py)

In [9]:
# Load company and worker information from the sample JSON
with open("/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/data/inputs/02_sample_prototype.json", "r", encoding="utf-8") as f:
    sample_data = json.load(f)

company_info = json.dumps(sample_data["company"], ensure_ascii=False, indent=2)
worker_info_dict = sample_data["worker"]
worker_info = json.dumps(worker_info_dict, ensure_ascii=False, indent=2)
constraints = sample_data["company"]["comment"]

# Set worker info data for tools
set_worker_info_data(worker_info_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/data/inputs/02_sample_prototype.json'

In [9]:
WorkerShift

module.shift_creator.WorkerShift

In [10]:
def test_create_shift_from_natural_language():
    """Tests that a simple natural language input can be correctly converted into a WorkerShift object."""
    # Arrange
    user_input = ""
    expected_shift = Shift(shift_id=9001, day="2025-07-20", start="10:00", finish="18:00")
    expected_worker_shift = WorkerShift(shifts={"shift_1": expected_shift})

    # Act
    # The agent_executor returns a dictionary with 'output' key
    result = chain.invoke({
        "input": user_input,
        "company_info": company_info,
        "worker_info": worker_info,
        "constraints": constraints
    })
    actual_worker_shift = WorkerShift(**json.loads(result["output"]))

    # Assert
    assert actual_worker_shift == expected_worker_shift

def test_create_multiple_shifts_from_natural_language():
    """Tests that a natural language input requesting multiple shifts can be correctly converted into a WorkerShift object."""
    # Arrange
    user_input = "高橋さんに2025-07-21の9時から17時までシフトを入れてください。IDは9002です。それから、2025-07-22の13時から22時までシフトを入れてください。IDは9003です。"
    expected_shift_1 = Shift(shift_id=9002, day="2025-07-21", start="09:00", finish="17:00")
    expected_shift_2 = Shift(shift_id=9003, day="2025-07-22", start="13:00", finish="22:00")
    expected_worker_shift = WorkerShift(shifts={"shift_1": expected_shift_1, "shift_2": expected_shift_2})

    # Act
    result = chain.invoke({
        "input": user_input,
        "company_info": company_info,
        "worker_info": worker_info,
        "constraints": constraints
    })
    actual_worker_shift = WorkerShift(**json.loads(result["output"]))

    # Assert
    assert actual_worker_shift == expected_worker_shift

def test_shift_creation_with_newbie_veteran_constraint():
    """Tests that the LLM considers the constraint: '新人の田中さんは、必ずベテランの誰かと組ませてください。'"""
    # Arrange
    user_input = "田中さんに2025-07-20の10時から18時までシフトを入れてください。IDは9004です。"

    # Extract veteran worker shifts from sample_data
    veteran_shifts = []
    for worker_key, worker_data in sample_data["worker"].items():
        if worker_data["experient"] == "ベテラン":
            for shift_key, shift_data in worker_data["shift"].items():
                veteran_shifts.append(Shift(**shift_data))

    # Act
    result = chain.invoke({
        "input": user_input,
        "company_info": company_info,
        "worker_info": worker_info,
        "constraints": constraints
    })
    actual_worker_shift = WorkerShift(**json.loads(result["output"]))

    # Assert
    assert len(actual_worker_shift.shifts) > 0
    tanaka_shift = list(actual_worker_shift.shifts.values())[0] # Assuming only one shift is generated for Tanaka

    # Check if Tanaka's shift overlaps with any veteran's shift
    overlap_found = False
    for veteran_shift in veteran_shifts:
        if tanaka_shift.overlaps_with(veteran_shift):
            overlap_found = True
            break
    assert overlap_found, "Tanaka's shift does not overlap with any veteran's shift, violating the constraint."

In [11]:
test_create_shift_from_natural_language()



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [1]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

In [2]:
from dotenv import load_dotenv
import os
import json

In [3]:
load_dotenv()

if os.getenv('GOOGLE_API_KEY'):
  google_api_key = os.getenv('GOOGLE_API_KEY')
else:
  print("Google API key not found.")

In [5]:
from langchain.chat_models import init_chat_model

# Returns a langchain_google_vertexai.ChatVertexAI instance.
gemini_15 = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai", temperature=0
)

# Since all model integrations implement the ChatModel interface, you can use them in the same way.
print("Gemini 1.5: " + gemini_15.invoke("what's your name").content + "\n")

Gemini 1.5: I am a large language model, trained by Google.



In [6]:
API_KEY = google_api_key
llm = init_chat_model(
    model="gemini-2.5-flash",
    model_provider='google_genai',
    temperature=0
)

ai_responce = llm.invoke("LangChainについて教えて")
print (ai_responce)

Unexpected argument 'temperture' provided to ChatGoogleGenerativeAI. Did you mean: 'temperature'?
/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/.venv/lib/python3.13/site-packages/langchain/chat_models/base.py:322: UserWarning: WARNING! temperture is not default parameter.
                temperture was transferred to model_kwargs.
                Please confirm that temperture is what you intended.
  return _init_chat_model_helper(


content='LangChainは、**大規模言語モデル（LLM）を活用したアプリケーションを効率的に開発するためのオープンソースフレームワーク**です。LLMの強力な能力を最大限に引き出し、より複雑で実用的なアプリケーションを構築するのを支援します。\n\n### LangChainが解決する課題\n\nLLMは単体でも素晴らしい能力を持っていますが、以下のような課題があります。\n\n1.  **外部データへのアクセス:** LLMは学習データ以降の情報を知らないため、最新情報や特定の組織の内部データにアクセスできません。\n2.  **記憶力（ステートフルネス）:** 会話の履歴を覚えておらず、文脈を維持した連続的な対話が苦手です。\n3.  **アクションの実行:** 外部ツール（検索エンジン、API、データベースなど）を呼び出して特定のアクションを実行する能力がありません。\n4.  **複雑なタスクの分解:** 複雑な要求を小さなステップに分解し、それぞれにLLMやツールを適用して解決する推論能力が限られています。\n\nLangChainはこれらの課題を解決し、LLMを「脳」として、外部の「目」「耳」「手足」と連携させる役割を果たします。\n\n### LangChainの主要なコンポーネント\n\nLangChainは、LLMアプリケーション開発に必要な様々なモジュールを提供しており、これらを組み合わせて利用します。\n\n1.  **Models (モデル)**:\n    *   様々なLLM（OpenAIのGPTシリーズ、AnthropicのClaude、Hugging Faceのオープンソースモデルなど）への統一されたインターフェースを提供します。\n    *   テキスト生成モデル（LLMs）、チャットモデル（ChatModels）、埋め込みモデル（Embeddings）などがあります。\n\n2.  **Prompts (プロンプト)**:\n    *   LLMへの指示（プロンプト）を管理・最適化するための機能です。\n    *   プロンプトテンプレート、チャット履歴の管理、出力解析などが含まれます。\n\n3.  **Chains (チェーン)**:\n    *   複数のコンポーネント（LLMの呼び出し、プロンプト処理

In [21]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=30,
    max_retries=2,
)

def call_gemini_model(system_instruction: str, user_content: str):
    """Calls the Gemini model with a system instruction and user content."""
    messages = [
        ("system", system_instruction),
        ("human", user_content)
    ]
    print(f"ヘルパー関数が呼び出されました．\n{messages}")
    for chunk in llm.stream(messages):
        print(chunk.content, end="")
    print(f"llmをinvokeし終わりました・")

In [22]:
def create_shift_draft_tool(full_json_input: str) -> str:
    """
    Creates a draft for a new shift based on a comprehensive JSON input.
    This tool analyzes the full company and worker data to propose a new shift,
    prioritizing the worker's requested shifts.

    Args:
        full_json_input: A string containing the full JSON data with company info,
                         worker details, and constraints.

    Returns:
        A JSON string representing the newly created draft shift for one worker.
    """
    system_instruction = """You are an expert shift creator. Your task is to create a single new shift draft based on the provided JSON data.
    The JSON contains all company information, a list of all workers with their details, their existing shift requests, and overall constraints.

    Your primary goal is to satisfy the workers' shift requests as much as possible. The input contains the shifts they have *already requested*.
    You must analyze the input and infer which single shift should be created next. Often, this means creating a shift for a worker who has requested one but it has not been formalized yet, or fulfilling a specific need based on the constraints (like ensuring a rookie is covered).

    Based on all the provided information, propose ONE new shift for ONE worker.

    Output only the JSON for the newly created shift. Do not include any other text, explanations, or markdown formatting.
    The output format must be a JSON object with 'worker_name' and 'shift' keys.
    Example Output:
    { "worker_name": "高橋 康成", "shift": { "shift_id": 9001, "day": "2025-07-25", "start": "10:00", "finish": "18:00" } }
    """
    user_content = full_json_input
    print(f"ユーザーコンテントが定義されました．\n{user_content}")
    return call_gemini_model(system_instruction, user_content)

In [23]:
import json
import os
def load_dummy_input_json():
    with open("/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/data/inputs/03_sample.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return json.dumps(data, ensure_ascii=False)

if __name__ == "__main__":
    print("--- Testing create_shift_draft_tool with dummy data ---")

    if not os.getenv('GOOGLE_API_KEY'):
        print("WARNING: GOOGLE_API_KEY environment variable not set. Tool might fail if it makes actual API calls.")
    dummy_data = load_dummy_input_json()

    try:
        print("Invoking create_shift_draft_tool...")
        result_json = create_shift_draft_tool(dummy_data)
        print(result_json)
        result_data = json.loads(result_json)

    except json.JSONDecodeError as e:
        print(f"ERROR: Tool did not return a valid JSON string. {e}")
        print(f"Invalid JSON: {result_json}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during tool execution or validation: {e}")

    print("\n--- Test Finished ---")

--- Testing create_shift_draft_tool with dummy data ---
Invoking create_shift_draft_tool...
ユーザーコンテントが定義されました．
{"company": {"company_id": 1, "open_time": "09:00", "close_time": "23:00", "rest_day": {"rest_day_1": "2025-07-01", "rest_day_2": "2025-07-08", "rest_day_3": "2025-07-15"}, "labor_cost": 670000, "comment": "佐藤さんと鈴木さんはできるだけ同じシフトに入れないでください。新人の田中さんは、必ずベテランの誰かと組ませてください。"}, "worker": {"worker_1": {"user_id": 101, "name": "高橋 康成", "evaluate": 5, "position": "キッチン, ホール, レジ", "experient": "ベテラン", "post": "正社員", "hour_pay": 1500, "shift": {"shift_1": {"shift_id": 1001, "day": "2025-07-02", "start": "11:00", "finish": "20:00"}, "shift_2": {"shift_id": 1002, "day": "2025-07-03", "start": "11:00", "finish": "20:00"}, "shift_3": {"shift_id": 1003, "day": "2025-07-04", "start": "14:00", "finish": "23:00"}, "shift_4": {"shift_id": 1004, "day": "2025-07-07", "start": "09:00", "finish": "18:00"}, "shift_5": {"shift_id": 1005, "day": "2025-07-10", "start": "11:00", "finish": "20:00"}, "shift_6"

KeyboardInterrupt: 

In [15]:
import json
import os
from unittest.mock import patch
from module.shift_creator import create_shift_draft_tool, EditShiftSchedule, EditShiftEntry, eval_shift_tool, ShiftEvaluation, modify_shift_tool

ImportError: attempted relative import beyond top-level package

In [17]:
# Load dummy input JSON data
def load_dummy_input_json():
    with open("/Users/takahashikaisei/mydir/shift-agent/backend/app/service/agent/data/inputs/02_sample.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    return json.dumps(data, ensure_ascii=False)



# Set GOOGLE_API_KEY for direct execution
if not os.getenv('GOOGLE_API_KEY'):
    print("WARNING: GOOGLE_API_KEY environment variable not set. Tool might fail if it makes actual API calls.")

dummy_data = load_dummy_input_json()

draft_json = create_shift_draft_tool.invoke(dummy_data)

In [18]:
file_path = "../data/outputs/draft_json.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(draft_json)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/draft_json.txt' に正常に保存されました。


In [19]:
with open("../data/outputs/draft_json.txt", "r", encoding='utf-8') as f:
    draft_json = f.read()
print(draft_json)

{
  "edit_shift": [
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "17:00",
      "finish_time": "23:00"
    },
    {
      "user_id": "田中 昌弘",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "18:00",
      "finish_time": "22:00"
    },
    {
      "user_id": "吉田 正孝",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "11:00",
      "finish_time": "19:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "11:00",
      "finish_time": "20:00"
    },
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "18:00"
    },
    {
      "user_id": "山本 良伸",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "19:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-03",
      "start_tim

In [20]:
# 方法1: 構造化JSONとして渡す（推奨）
input_data = json.dumps({
    "employee_preferences": dummy_data,  # 従業員の希望データ
    "shift_draft": draft_json                          # シフトドラフト
})
eval_result_json = eval_shift_tool.invoke(input_data)

# # 方法2: 単純な文字列として渡す
# combined_input = f"Employee Preferences: {employee_preferences_json}\nShift Draft: {draft_json}"
# eval_result_json = eval_shift_tool.invoke(combined_input) draft from previous step

In [21]:
file_path = "../data/outputs/eval_result_json.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(eval_result_json)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/eval_result_json.txt' に正常に保存されました。


In [22]:
with open("../data/outputs/eval_result_json.txt", "r", encoding='utf-8') as f:
    eval_result_json = f.read()
print(eval_result_json)

{
    "quantitative_score": 85,
    "feedback_japanese": "初期スコア100点から、以下の点が減点されました。\n- 人件費は予算内に収まっています (提案シフトの人件費は363750円でした)。\n- レジ人員の不足：-3点 (2025-07-02 09:00-11:00にレジ担当が0人でした。)\n- キッチン人員の不足：-3点 (2025-07-02 09:00-11:00にキッチン担当が0人でした。)\n- ホール人員の不足：-3点 (2025-07-02 09:00-11:00にホール担当が0人でした。)\n- レジ人員の不足：-3点 (2025-07-04 09:00-14:00にレジ担当が0人でした。)\n- キッチン人員の不足：-3点 (2025-07-04 09:00-14:00にキッチン担当が0人でした。)\n- 従業員のシフト希望の未考慮：-0.4点\n- 鈴木 一浪さんの2025-07-03 17:00-23:00の希望シフトが叶っていません。\n- 鈴木 一浪さんの2025-07-09 11:00-20:00の希望シフトが叶っていません。\n\n【努力目標に関するフィードバック】\n佐藤さんと鈴木さんが同じシフトに入ることはありませんでした。\n\n【必須要件に関するフィードバック（減点対象外だが重要）】\n新人の田中さんは全てのシフトでベテランと組んでいます。"
}


In [23]:
import json

In [24]:
# 方法1: 構造化JSONとして渡す（推奨）
input_data = json.dumps({
    "shift_draft": draft_json,
    "evaluation_result": eval_result_json
})
modified_json = modify_shift_tool.invoke(input_data)

# # 方法2: 単純な文字列として渡す
# combined_input = f"Shift Draft: {draft_json}\nEvaluation Result: {eval_result_json}"
# modified_json = modify_shift_tool.invoke(combined_input)

In [25]:
modified_json

'{\n  "edit_shift": [\n    {\n      "user_id": "佐藤 照明",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "17:00",\n      "finish_time": "23:00"\n    },\n    {\n      "user_id": "田中 昌弘",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "18:00",\n      "finish_time": "22:00"\n    },\n    {\n      "user_id": "吉田 正孝",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "09:00",\n      "finish_time": "17:00"\n    },\n    {\n      "user_id": "高橋 康成",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "11:00",\n      "finish_time": "20:00"\n    },\n    {\n      "user_id": "佐藤 照明",\n      "company_id": 1,\n      "day": "2025-07-03",\n      "start_time": "11:00",\n      "finish_time": "18:00"\n    },\n    {\n      "user_id": "山本 良伸",\n      "company_id": 1,\n      "day": "2025-07-03",\n      "start_time": "11:00",\n      "finish_time": "19:00"\n    },\n    {\n      "user_id": "高橋 康成",\n      "company_id":

In [26]:
file_path = "../data/outputs/modified_json.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(modified_json)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/modified_json.txt' に正常に保存されました。


In [27]:
with open("../data/outputs/modified_json.txt", "r", encoding='utf-8') as f:
    modified_json = f.read()
print(modified_json)

{
  "edit_shift": [
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "17:00",
      "finish_time": "23:00"
    },
    {
      "user_id": "田中 昌弘",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "18:00",
      "finish_time": "22:00"
    },
    {
      "user_id": "吉田 正孝",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "09:00",
      "finish_time": "17:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "11:00",
      "finish_time": "20:00"
    },
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "18:00"
    },
    {
      "user_id": "山本 良伸",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "19:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-03",
      "start_tim

In [38]:
# 方法1: 構造化JSONとして渡す（推奨）
input_data = json.dumps({
    "employee_preferences": dummy_data,  # 従業員の希望データ
    "shift_draft": modified_json                          # シフトドラフト
})
eval_result_json_2 = eval_shift_tool.invoke(input_data)

In [39]:
eval_result_json_2

'{\n    "quantitative_score": 73,\n    "feedback_japanese": "初期スコア100点から、以下の点が減点されました。\\n- ホール人員の不足：-3点 (2025-07-02 09:00-10:00にホール担当が0人でした。)\\n- レジ人員の不足：-3点 (2025-07-02 09:00-10:00にレジ担当が0人でした。)\\n- ホール人員の不足：-3点 (2025-07-02 10:00-11:00にホール担当が0人でした。)\\n- レジ人員の不足：-3点 (2025-07-02 10:00-11:00にレジ担当が0人でした。)\\n- キッチン人員の不足：-3点 (2025-07-04 09:00-10:00にキッチン担当が0人でした。)\\n- キッチン人員の不足：-3点 (2025-07-04 10:00-11:00にキッチン担当が0人でした。)\\n- キッチン人員の不足：-3点 (2025-07-04 11:00-12:00にキッチン担当が0人でした。)\\n- キッチン人員の不足：-3点 (2025-07-04 12:00-13:00にキッチン担当が0人でした。)\\n- キッチン人員の不足：-3点 (2025-07-04 13:00-14:00にキッチン担当が0人でした。)\\n- 人件費は予算内(670000円)に収まっています (総人件費 599000円)。\\n- 従業員のシフト希望の未考慮：-0.2点\\n- 吉田 正孝さんの2025-07-02 11:00-19:00のシフト希望が叶っていません。\\n- 会社規則: 佐藤さんと鈴木さんが2025-07-03に同じ時間帯(17:00-18:00)でシフトに入っています。できるだけ避けるよう指示がありました。\\n- 会社規則: 佐藤さんと鈴木さんが2025-07-09に同じ時間帯(17:00-20:00)でシフトに入っています。できるだけ避けるよう指示がありました。\\n- 会社規則の違反:\\n- 新人の大谷 昌平さんが2025-07-06 18:00-22:00のシフトでベテランと組んでいません。\\n- 新人の大谷 昌平さんが2025-07-11 17:00-22:00のシフトでベテランと組んでいません。"\n}'

In [40]:
file_path = "../data/outputs/eval_result_json_2.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(eval_result_json_2)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/eval_result_json_2.txt' に正常に保存されました。


In [41]:
# 方法1: 構造化JSONとして渡す（推奨）
input_data = json.dumps({
    "shift_draft": modified_json,
    "evaluation_result": eval_result_json_2
})
modified_json_2 = modify_shift_tool.invoke(input_data)

In [42]:
modified_json_2

'{\n  "edit_shift": [\n    {\n      "user_id": "佐藤 照明",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "17:00",\n      "finish_time": "23:00"\n    },\n    {\n      "user_id": "田中 昌弘",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "18:00",\n      "finish_time": "22:00"\n    },\n    {\n      "user_id": "吉田 正孝",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "11:00",\n      "finish_time": "19:00"\n    },\n    {\n      "user_id": "高橋 康成",\n      "company_id": 1,\n      "day": "2025-07-02",\n      "start_time": "09:00",\n      "finish_time": "18:00"\n    },\n    {\n      "user_id": "佐藤 照明",\n      "company_id": 1,\n      "day": "2025-07-03",\n      "start_time": "11:00",\n      "finish_time": "17:00"\n    },\n    {\n      "user_id": "山本 良伸",\n      "company_id": 1,\n      "day": "2025-07-03",\n      "start_time": "11:00",\n      "finish_time": "19:00"\n    },\n    {\n      "user_id": "高橋 康成",\n      "company_id":

In [43]:
file_path = "../data/outputs/modified_json_2.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(modified_json_2)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/modified_json_2.txt' に正常に保存されました。


In [44]:
print(modified_json_2)

{
  "edit_shift": [
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "17:00",
      "finish_time": "23:00"
    },
    {
      "user_id": "田中 昌弘",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "18:00",
      "finish_time": "22:00"
    },
    {
      "user_id": "吉田 正孝",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "11:00",
      "finish_time": "19:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-02",
      "start_time": "09:00",
      "finish_time": "18:00"
    },
    {
      "user_id": "佐藤 照明",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "17:00"
    },
    {
      "user_id": "山本 良伸",
      "company_id": 1,
      "day": "2025-07-03",
      "start_time": "11:00",
      "finish_time": "19:00"
    },
    {
      "user_id": "高橋 康成",
      "company_id": 1,
      "day": "2025-07-03",
      "start_tim

In [45]:
# 方法1: 構造化JSONとして渡す（推奨）
input_data = json.dumps({
    "employee_preferences": dummy_data,  # 従業員の希望データ
    "shift_draft": modified_json_2                          # シフトドラフト
})
eval_result_json_3 = eval_shift_tool.invoke(input_data)

In [46]:
eval_result_json_3

'{\n  "quantitative_score": 99.4,\n  "feedback_japanese": "初期スコア100点から、以下の点が減点されました。\\n- 従業員のシフト希望の未考慮：-0.6点\\n- 高橋 康成さんの2025-07-02 11:00-20:00の希望シフトが叶っていません。\\n- 佐藤 照明さんの2025-07-03 11:00-18:00の希望シフトが叶っていません。\\n- 鈴木 一浪さんの2025-07-09 11:00-20:00の希望シフトが叶っていません。\\n\\nその他の評価：\\n- 人件費は予算内に収まっています。(総人件費: 660000円, 予算: 670000円)\\n- ホール、レジ、キッチンの最低人員配置は満たされています。\\n\\n会社規則に関する評価：\\n- 会社規則（努力目標）：佐藤さんと鈴木さんは同じシフトに入っていません。\\n- 会社規則：新人の田中さんは全てのシフトでベテランと組んでいます。\\n- 会社規則：新人の大谷さんがベテランと組んでいないシフトがあります。\\n- 2025-07-06 18:00-22:00\\n- 2025-07-11 17:00-22:00"\n}'

In [47]:
print(eval_result_json_3)

{
  "quantitative_score": 99.4,
  "feedback_japanese": "初期スコア100点から、以下の点が減点されました。\n- 従業員のシフト希望の未考慮：-0.6点\n- 高橋 康成さんの2025-07-02 11:00-20:00の希望シフトが叶っていません。\n- 佐藤 照明さんの2025-07-03 11:00-18:00の希望シフトが叶っていません。\n- 鈴木 一浪さんの2025-07-09 11:00-20:00の希望シフトが叶っていません。\n\nその他の評価：\n- 人件費は予算内に収まっています。(総人件費: 660000円, 予算: 670000円)\n- ホール、レジ、キッチンの最低人員配置は満たされています。\n\n会社規則に関する評価：\n- 会社規則（努力目標）：佐藤さんと鈴木さんは同じシフトに入っていません。\n- 会社規則：新人の田中さんは全てのシフトでベテランと組んでいます。\n- 会社規則：新人の大谷さんがベテランと組んでいないシフトがあります。\n- 2025-07-06 18:00-22:00\n- 2025-07-11 17:00-22:00"
}


In [48]:
file_path = "../data/outputs/eval_result_json_3.txt"

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(eval_result_json_3)
print(f"データが '{file_path}' に正常に保存されました。")

データが '../data/outputs/eval_result_json_3.txt' に正常に保存されました。
